# Problem Statement

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business
goal. To reduce customer churn, telecom companies need to predict which customers are at high risk of churn. In this project, you will analyze customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn.

In this competition, your goal is to build a machine learning model that is able to predict churning customers based on the features provided for their usage.

# Data preparation and cleansing
## Import neccessary packages

Although in Python we can import packages anywhere we want, as long as it's prior the use, it'd be better to keep all of neccessary packages in one place. It's easer for reference.

In [1]:
%pip install catboost xgboost optuna pydotplus six

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## i love this function, it helps me to determine if a column is numeric or not. Why didn't I find it earlier
from pandas.api.types import is_numeric_dtype 

import warnings
import optuna
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV


from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
# from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

from sklearn.metrics import plot_roc_curve


from sklearn.metrics import (accuracy_score, log_loss, confusion_matrix)

# I would like to see all the detail info in info() or describe()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Exploratory Data Analysis
### Import dataset from current working folder


In [3]:
df = pd.read_csv("train (1).csv")
df.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,7.527,48.58,124.38,1.29,32.24,96.68,2.33,0.00,0.0,0.0,0.00,0.0,0.00,2.23,0.00,0.28,5.29,16.04,2.33,0.00,0.00,0.00,0.00,0.00,0.00,7.53,16.04,2.61,46.34,124.38,1.01,18.75,80.61,0.0,0.00,0.0,0.0,0.0,0.0,0.0,65.09,204.99,1.01,0.0,0.0,0.0,8.20,0.63,0.00,0.38,0.0,0.0,81.21,221.68,3.63,2.43,3.68,7.79,0.83,21.08,16.91,0.00,0.00,0.00,3.26,24.76,24.71,0.00,7.61,0.21,7.46,19.96,14.96,0.0,0.0,0.0,0.0,0.0,0.0,7.46,27.58,15.18,11.84,53.04,40.56,0.0,0.0,0.66,0.0,0.0,0.0,1.11,0.69,0.00,3,2,2,77,65,10,65,65,10,6/22/2014,7/10/2014,8/24/2014,65,65,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.0,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,42.953,0.00,0.00,0.00,0.00,25.99,30.89,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22.01,29.79,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,30.73,31.66,0.00,0.0,0.0,0.00,30.73,31.66,1.68,19.09,10.53,1.41,18.68,11.09,0.35,1.66,3.40,3.44,39.44,25.03,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,3.44,39.44,25.04,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.00,0.00,3,4,5,0,145,50,0,145,50,6/12/2014,7/10/2014,8/26/2014,0,0,0,NaN,7/8/2014,NaN,NaN,1.0,NaN,NaN,145.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,Na

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Columns: 172 entries, id to churn_probability
dtypes: float64(135), int64(28), object(9)
memory usage: 91.9+ MB


In [5]:
df.shape

(69999, 172)

In [6]:
df.describe()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
count,69999.000000,69999.0,69297.0,69297.0,69297.0,69999.000000,69999.000000,69999.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.0,67312.0,66296.0,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,69999.000000,69999.000000,69999.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.0,67312.0,66296.0,67231.000000,67312.000000,66296.000000,69999.000000,69999.000000,69999.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,67231.000000,67312.000000,66296.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,17568.000000,17865.000000,18417.000000,17568.000000,17865.000000,18417.000000,17568.000000,17865.000000,18417.000000,17568.000000,17865.000000,18417.000000,17568.000000,17865.000000,18417.000000,69999.000000,69999.000000

In [7]:
df.describe(include='object')

,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8
count,69999,69600,69266,68898,68765,67538,17568,17865,18417
unique,1,1,1,30,31,31,30,31,31
top,6/30/2014,7/31/2014,8/31/2014,6/30/2014,7/31/2014,8/31/2014,6/30/2014,7/31/2014,8/31/2014
freq,69999,69600,69266,11880,12206,10324,1317,1282,1388


### Handling nulls value

In [8]:
total = df.shape[0]

## using for loop to determine missing value in each columns in the dataset
missing_columns = [col for col in df.columns if df[col].isnull().sum() >= 0]
missing_columns

['id',
 'circle_id',
 'loc_og_t2o_mou',
 'std_og_t2o_mou',
 'loc_ic_t2o_mou',
 'last_date_of_month_6',
 'last_date_of_month_7',
 'last_date_of_month_8',
 'arpu_6',
 'arpu_7',
 'arpu_8',
 'onnet_mou_6',
 'onnet_mou_7',
 'onnet_mou_8',
 'offnet_mou_6',
 'offnet_mou_7',
 'offnet_mou_8',
 'roam_ic_mou_6',
 'roam_ic_mou_7',
 'roam_ic_mou_8',
 'roam_og_mou_6',
 'roam_og_mou_7',
 'roam_og_mou_8',
 'loc_og_t2t_mou_6',
 'loc_og_t2t_mou_7',
 'loc_og_t2t_mou_8',
 'loc_og_t2m_mou_6',
 'loc_og_t2m_mou_7',
 'loc_og_t2m_mou_8',
 'loc_og_t2f_mou_6',
 'loc_og_t2f_mou_7',
 'loc_og_t2f_mou_8',
 'loc_og_t2c_mou_6',
 'loc_og_t2c_mou_7',
 'loc_og_t2c_mou_8',
 'loc_og_mou_6',
 'loc_og_mou_7',
 'loc_og_mou_8',
 'std_og_t2t_mou_6',
 'std_og_t2t_mou_7',
 'std_og_t2t_mou_8',
 'std_og_t2m_mou_6',
 'std_og_t2m_mou_7',
 'std_og_t2m_mou_8',
 'std_og_t2f_mou_6',
 'std_og_t2f_mou_7',
 'std_og_t2f_mou_8',
 'std_og_t2c_mou_6',
 'std_og_t2c_mou_7',
 'std_og_t2c_mou_8',
 'std_og_mou_6',
 'std_og_mou_7',
 'std_og_mou_8',
 

In [9]:
total = df.shape[0]

## using for loop to determine missing value in each columns in the dataset
missing_columns = [col for col in df.columns if df[col].isnull().sum() >= 0]

## this list contains the final columns after dropping all-null-value columns
final_columns = []

## using for loop to calculate % of null values in each columns.
for col in missing_columns:
    null_count = df[col].isnull().sum()
    per = (null_count/total) * 100
    print(f"{col}: {null_count} ({round(per, 3)}%)")
    
    ## if null percentage is less than 10%, those columns are kept in the final list
    ## detail rationale will be described in the result and below text.
    if per < 10:
        final_columns.append(col)

id: 0 (0.0%)
circle_id: 0 (0.0%)
loc_og_t2o_mou: 702 (1.003%)
std_og_t2o_mou: 702 (1.003%)
loc_ic_t2o_mou: 702 (1.003%)
last_date_of_month_6: 0 (0.0%)
last_date_of_month_7: 399 (0.57%)
last_date_of_month_8: 733 (1.047%)
arpu_6: 0 (0.0%)
arpu_7: 0 (0.0%)
arpu_8: 0 (0.0%)
onnet_mou_6: 2768 (3.954%)
onnet_mou_7: 2687 (3.839%)
onnet_mou_8: 3703 (5.29%)
offnet_mou_6: 2768 (3.954%)
offnet_mou_7: 2687 (3.839%)
offnet_mou_8: 3703 (5.29%)
roam_ic_mou_6: 2768 (3.954%)
roam_ic_mou_7: 2687 (3.839%)
roam_ic_mou_8: 3703 (5.29%)
roam_og_mou_6: 2768 (3.954%)
roam_og_mou_7: 2687 (3.839%)
roam_og_mou_8: 3703 (5.29%)
loc_og_t2t_mou_6: 2768 (3.954%)
loc_og_t2t_mou_7: 2687 (3.839%)
loc_og_t2t_mou_8: 3703 (5.29%)
loc_og_t2m_mou_6: 2768 (3.954%)
loc_og_t2m_mou_7: 2687 (3.839%)
loc_og_t2m_mou_8: 3703 (5.29%)
loc_og_t2f_mou_6: 2768 (3.954%)
loc_og_t2f_mou_7: 2687 (3.839%)
loc_og_t2f_mou_8: 3703 (5.29%)
loc_og_t2c_mou_6: 2768 (3.954%)
loc_og_t2c_mou_7: 2687 (3.839%)
loc_og_t2c_mou_8: 3703 (5.29%)
loc_og_mou_6: 

From the result above, we can see there are two type of missing values:
* Missing less than 5% of data
* Missing about 75% of data

I think it's safe to remove all columns missing 75% of data. From now on, those columns are removed.

In [10]:
## from my past experiences, we shouldn't overwrite the old data frame.
## if we really want to, just create a new data frame for each step.
new_df = df[final_columns]

In [11]:
new_df.shape

(69999, 142)

18% columns are dropped, perhaps it's not a wise choice now. After the data model is built, if the result is not good, we'll get back to this section again. Remaining columns are examined columns type. Based on each columns type, different data imputing methods are required.

In [12]:
columns_name = new_df.columns
columns_type = new_df.dtypes

records = []
for i in range(len(columns_name)):
    records.append([columns_name[i], columns_type[i]])
    
new_df_meta_data = pd.DataFrame(records, columns=['column_name', 'column_type'])

pd.pivot_table(new_df_meta_data
               , index = "column_type"
               , aggfunc=['count'] 
              )

,count
,column_name
column_type,
int64,28
float64,108
object,6


Great, there are only float and object. 
* int64, float64: will be imputed using median.
* object: will be imputed using mode.

#### Imputing numeric columns using median

In [13]:
numeric_columns = new_df.select_dtypes(include=["float64", "int64"]).columns.tolist()

for col in numeric_columns:
    new_df[col].fillna(new_df[col].median(), inplace=True)

In [14]:
new_df = new_df[numeric_columns]

### Outlier treatment
#### Dropping outliers using quantiles.
Extreme values are considered outliers and should be removed before hand. To make sure, we should draw histograms to see if there are any outliers

In [ ]:
a = int(round(len(numeric_columns) / 2, 0)) 

f, axes = plt.subplots(a, 2, figsize=(20,300), sharex=False)
idx, idy = 0, 0

for i in range(1, len(new_df.columns)):
    if is_numeric_dtype(new_df.iloc[:,i]):
        plot = sns.distplot(x=new_df.iloc[:,i], color="skyblue", ax=axes[idx, idy])
        plot.set(xlabel=new_df.columns[i])
        idx += idy
        idy = int(not idy)
plt.show()

Base on above charts, there are clearly some columns that can be dropped, because they have only one value, which is 0:
* std_og_t2o_mou
* loc_ic_t2o_mou
* std_og_t2c_mou_6
* std_og_t2c_mou_7
* std_og_t2c_mou_8
* std_ic_t2o_mou_6
* std_ic_t2o_mou_7
* std_ic_t2o_mou_8

In [ ]:
## creating a new variables for columns with all zero values
columns_zero_values = ['std_og_t2o_mou'
                       , 'loc_ic_t2o_mou'
                       , 'std_og_t2c_mou_6'
                       , 'std_og_t2c_mou_7'
                       , 'std_og_t2c_mou_8'
                       , 'std_ic_t2o_mou_6'
                       , 'std_ic_t2o_mou_7'
                       , 'std_ic_t2o_mou_8'
                      ]

In [ ]:
## drop those columns from final dataset

new_df.drop(columns_zero_values, inplace=True, axis=1)

According to all above histograms, our data is right skew, as a matter of fact, it's common sense for transaction data. To remove outliers, we can just remove data from the right tail. 

In [ ]:
# numeric_columns variable is reused from previous section, which is imputing numeric columns using median.
numeric_columns = new_df.select_dtypes(include=["float64", "int64"]).columns.tolist()

# initially, I want to remove 5% data from the right, but it will remove more than 55% of data.
# after a few attemps, threshold 1% is good enough.
max_threshold = 0.99
min_threshod = 0

col_with_min_threshold = new_df[numeric_columns].quantile(min_threshod)
col_with_max_threshold = new_df[numeric_columns].quantile(max_threshold)

# calculating the range from min to max
in_use_range = col_with_max_threshold - col_with_min_threshold

# all data beyond that range are removed. 
# new dataset after_outliers is created to reserve new_df (containing data after imputing data)
after_outliers = new_df[~((new_df[numeric_columns] < (col_with_min_threshold - (1-min_threshod) * in_use_range)) |(new_df[numeric_columns] > (col_with_max_threshold + (1-min_threshod) * in_use_range))).any(axis=1)]


In [ ]:
print("Shape of data before removing outliers: ", new_df.shape)
print("Shape of data before removing outliers: ", after_outliers.shape)

In [ ]:
# a = after_outliers.apply(pd.Series.nunique) / after_outliers.shape[0]

# print("Min percentage of unique values:", a.min())
# print("First quantile percentage of unique values:", a.quantile(0.25))
# print("Median percentage of unique values:", a.quantile(0.5))
# print("Mean percentage of unique values:", a.quantile(0.5))
# print("Third quantile percentage of unique values:", a.quantile(0.75))
# print("Max percentage of unique values:", a.max())

In [ ]:
# less_unique_value_columns = a[a < a.quantile(0.25)].index

In [ ]:
# after_outliers.drop(less_unique_value_columns, inplace=True, axis=1)

In [ ]:
# after_outliers.shape

In [ ]:
# after_outliers.columns

# Model building
Following algorithms are attempted to see the best fit model:
* Logistic regression using Lasso for feature selection
* Decision tree
* Random forest
* Gradient Boosting
* XGBoosting

Whichever algorithm is chosen, the preparation steps are similar. Therefore, all those steps are prepared before hand.

## Spliting data to train and test set

In [ ]:
x = after_outliers.drop(["id", "circle_id", "churn_probability"], axis = 1)
y = after_outliers.churn_probability

In [ ]:
np.random.seed(1309) ## it's my favorite to use current date as seed :D

x_train, x_test, y_train, y_test = train_test_split(x.values, y.values, test_size=0.25, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(x
#                                                     , y
#                                                     , train_size=0.7
#                                                     , test_size = 0.3
#                                                     , random_state=100
#                                                    )

Let's check if the size of train and test dataset are 70% and 30%

In [ ]:
print("Size of X_train is: ", len(x_train))
print("Size of y_train is: ", len(y_train))

print("Size of X_test is: ", len(x_test))
print("Size of y_test is: ", len(y_test))

It seems the split work well

In [ ]:
## this variable contains all model evaluation result
model_evaluation = []

## List of algorithm
### Logistic regression

In [ ]:
lg_model = LogisticRegression(random_state = 42)
lg_model.fit(x_train, y_train)

In [ ]:
## Calculataing accuracy

print(f"Training Score: {lg_model.score(x_train , y_train)}")
print(f"Test Score: {lg_model.score(x_test , y_test)}")

In [ ]:
## Preparing confusion matrix

y_pred_lg = lg_model.predict(x_test)

cm_lg = confusion_matrix(y_test, y_pred_lg)
print('The confusion Matrix : \n',cm_lg)

FPR_lg = cm_lg[1,0] / (cm_lg[1,0] + cm_lg[1,1])
print("False positive rate", FPR_lg)

TPR_lg = cm_lg[0, 0] / (cm_lg[0, 0] + cm_lg[0, 1])
print("True positive rate", TPR_lg)

TNR_lg = cm_lg[1,1] / (cm_lg[1,0] + cm_lg[1,1])
print("True negative rate", TNR_lg)

In [ ]:
model_evaluation.append(["Logistic regression", lg_model.score(x_train , y_train), lg_model.score(x_test , y_test), TPR_lg, FPR_lg, TNR_lg])

### Decision tree

In [ ]:
def objective(trial):
    dt_max_depth = trial.suggest_int('dt_max_depth', 2, 32, log=False)
    dt_max_features = trial.suggest_int('dt_max_features', 2, 22, log=False)
    classifier_obj = DecisionTreeClassifier(max_features = dt_max_features, max_depth = dt_max_depth)
    classifier_obj.fit(x_train, y_train)
    accuracy = classifier_obj.score(x_test, y_test)
    return accuracy

In [ ]:
study_dt = optuna.create_study(direction='maximize')
study_dt.optimize(objective, n_trials=30)
print(study_dt.best_trial)

In [ ]:
## Calculataing accuracy

dt = DecisionTreeClassifier(max_features = study_dt.best_trial.params['dt_max_features'], max_depth = study_dt.best_trial.params['dt_max_depth'])
dt.fit(x_train, y_train)
print(f"Train Score: {dt.score(x_train, y_train)}")
print(f"Test Score: {dt.score(x_test, y_test)}")

In [ ]:
## Preparing confusion matrix

y_pred_dt = dt.predict(x_test)

cm_dt = confusion_matrix(y_test, y_pred_dt)
print('The confusion Matrix : \n',cm_dt)

FPR_dt = cm_dt[1,0] / (cm_dt[1,0] + cm_dt[1,1])
print("False positive rate", FPR_dt)

TPR_dt = cm_dt[0, 0] / (cm_dt[0, 0] + cm_dt[0, 1])
print("True positive rate", TPR_dt)

TNR_dt = cm_dt[1,1] / (cm_dt[1,0] + cm_dt[1,1])
print("True negative rate", TNR_dt)

In [ ]:
model_evaluation.append(["Decision tree", dt.score(x_train, y_train), dt.score(x_test, y_test), TPR_dt, FPR_dt, TNR_dt])

In [ ]:
fig = plt.figure(figsize = (30,12))
tree.plot_tree(dt, filled=True);
plt.show()

### Random Forest Classifier

In [ ]:
def objective(trial):
    rf_max_depth = trial.suggest_int('rf_max_depth', 2, 32, log=False)
    rf_max_features = trial.suggest_int('rf_max_features', 2, 22, log=False)
    rf_n_estimators = trial.suggest_int('rf_n_estimators', 3, 20, log=False)
    classifier_obj = RandomForestClassifier(max_features = rf_max_features, max_depth = rf_max_depth, n_estimators = rf_n_estimators)
    classifier_obj.fit(x_train, y_train)
    accuracy = classifier_obj.score(x_test, y_test)
    return accuracy

In [ ]:
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective, n_trials=30)
print(study_rf.best_trial)

In [ ]:
## Calculataing accuracy

rf = RandomForestClassifier(max_features = study_rf.best_trial.params['rf_max_features']
                            , max_depth = study_rf.best_trial.params['rf_max_depth']
                            , n_estimators = study_rf.best_trial.params['rf_n_estimators']
                           )
rf.fit(x_train, y_train)

print(f"Train Score: {rf.score(x_train, y_train)}")
print(f"Test Score: {rf.score(x_test, y_test)}")

In [ ]:
## Preparing confusion matrix

y_pred_rf = rf.predict(x_test)

cm_rf = confusion_matrix(y_test, y_pred_rf)
print('The confusion Matrix : \n',cm_rf)

FPR_rf = cm_rf[1,0] / (cm_rf[1,0] + cm_rf[1,1])
print("False positive rate", FPR_rf)

TPR_rf = cm_rf[0, 0] / (cm_rf[0, 0] + cm_rf[0, 1])
print("True positive rate", TPR_rf)

TNR_rf = cm_rf[1,1] / (cm_rf[1,0] + cm_rf[1,1])
print("True negative rate", TNR_rf)

In [ ]:
model_evaluation.append(["Random Forest Classifier", rf.score(x_train, y_train), rf.score(x_test, y_test), TPR_rf, FPR_rf, TNR_rf])

### Gradient Boosting

In [ ]:
clf = GradientBoostingClassifier(random_state=42)
clf.fit(x_train, y_train)

In [ ]:
## Calculataing accuracy

print(f"Training Score: {clf.score(x_train , y_train)}")
print(f"Test Score: {clf.score(x_test , y_test)}")

In [ ]:
## Preparing confusion matrix

y_pred_gb = clf.predict(x_test)

cm_gb = confusion_matrix(y_test, y_pred_gb)
print('The confusion Matrix : \n',cm_gb)

FPR_gb = cm_gb[1,0] / (cm_gb[1,0] + cm_gb[1,1])
print("False positive rate", FPR_gb)

TPR_gb = cm_gb[0, 0] / (cm_gb[0, 0] + cm_gb[0, 1])
print("True positive rate", TPR_gb)

TNR_gb = cm_gb[1,1] / (cm_gb[1,0] + cm_gb[1,1])
print("True negative rate", TNR_gb)

In [ ]:
model_evaluation.append(["Gradient Boosting", clf.score(x_train , y_train), clf.score(x_test , y_test), TPR_gb, FPR_gb, TNR_gb])

### XGBoosting

In [ ]:
xgb_model = XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(x_train, y_train)

In [ ]:
## Calculataing accuracy

print(f"Training Score: {xgb_model.score(x_train , y_train)}")
print(f"Test Score: {xgb_model.score(x_test , y_test)}")


In [ ]:
## Preparing confusion matrix

y_pred_xgb = xgb_model.predict(x_test)

cm_xgb = confusion_matrix(y_test, y_pred_xgb)
print('The confusion Matrix : \n',cm_xgb)

FPR_xgb = cm_xgb[1,0] / (cm_xgb[1,0] + cm_xgb[1,1])
print("False positive rate", FPR_xgb)

TPR_xgb = cm_xgb[0, 0] / (cm_xgb[0, 0] + cm_xgb[0, 1])
print("True positive rate", TPR_xgb)

TNR_xgb = cm_xgb[1,1] / (cm_xgb[1,0] + cm_xgb[1,1])
print("True negative rate", TNR_xgb)

In [ ]:
model_evaluation.append(["XGBoosting", xgb_model.score(x_train , y_train), xgb_model.score(x_test , y_test), TPR_xgb, FPR_xgb, TNR_xgb])

### AdaBoost

In [ ]:
ab_model = AdaBoostClassifier(random_state=42)
ab_model.fit(x_train, y_train)

In [ ]:
## Calculataing accuracy

print(f"Training Score: {ab_model.score(x_train , y_train)}")
print(f"Test Score: {ab_model.score(x_test , y_test)}")

In [ ]:
## Preparing confusion matrix

y_pred_xgb = ab_model.predict(x_test)

cm_xgb = confusion_matrix(y_test, y_pred_xgb)
print('The confusion Matrix : \n',cm_xgb)

FPR_xgb = cm_xgb[1,0] / (cm_xgb[1,0] + cm_xgb[1,1])
print("False positive rate", FPR_xgb)

TPR_xgb = cm_xgb[0, 0] / (cm_xgb[0, 0] + cm_xgb[0, 1])
print("True positive rate", TPR_xgb)

TNR_xgb = cm_xgb[1,1] / (cm_xgb[1,0] + cm_xgb[1,1])
print("True negative rate", TNR_xgb)

In [ ]:
model_evaluation.append(["AdaBoost", ab_model.score(x_train , y_train), ab_model.score(x_test , y_test), TPR_xgb, FPR_xgb, TNR_xgb])

## Model selection

In [ ]:
pd.DataFrame(model_evaluation
             , columns=['Algorithm', 'Training Score', 'Test score', 'True Positive rate', 'False Positive rate', 'True negative rate']
            ).sort_values(by=["Training Score", "Test score"], ascending=False) 

Gradient Boosting is chosen as the final model, because 
* the accuracy is stable across training and test dataset
* the false positive rate is the lowest
* the true negative rate is the highest ==> this model may be useful to detect user least likely to churn too.
* all algorithms give simiar true positive rate, so this metric is not in consideration.

# Model evaluation on actual data

In [ ]:
## actual dataset
test_df = pd.read_csv("test (1).csv")

## solution dataset
solution_df = pd.read_csv("solution (1).csv")

In [ ]:
test_df

In [ ]:
solution_df

## Data preparation on actual data

Because the final model doesn't use all columns in train dataset, the actual dataset needs to have similar columns too. 

In [ ]:
## create a new variable so-called actual dataset based on train dataset.
actual_df = test_df[x.columns.tolist()]

In [ ]:
actual_df

## Imputing nulls values on actual data

GradientBoostingClassifier doesn't accept missing values encoded as NaN natively. Those missing values must be imputed in the same treatment that we did for train dataset.

In [ ]:
numeric_columns = actual_df.select_dtypes(include=["float64", "int64"]).columns.tolist()

for col in numeric_columns:
    actual_df[col].fillna(actual_df[col].median(), inplace=True)

## Applying model on actual data

Here are following steps are done shortly
1. Predict on test data set (file name test (1).csv)
    * At the end of this step, we'll get churn prediction for each observation
2. Calculating confusion matrix using two columns
    * churn_probability in result dataset (file name solution (1).csv
    * churn prediction from step 1
3. Comparing the confusion matrix in this test dataset with the one we got in model building section
    * if there is similar, we can conclude our model is robust

In [ ]:
y_pred_actual = clf.predict(actual_df)

In [ ]:
## adding predicted data to solution dataset

solution_df['predicted_label'] = pd.Series(y_pred_actual)

In [ ]:
solution_df

In [ ]:
cm_gb_v2 = confusion_matrix(solution_df["churn_probability"].to_numpy(), y_pred_actual)
print('The confusion Matrix : \n',cm_gb_v2)

FPR_gb_v2 = cm_gb_v2[1,0] / (cm_gb_v2[1,0] + cm_gb_v2[1,1])
print("False positive rate", FPR_gb_v2)

TPR_gb_v2 = cm_gb_v2[0, 0] / (cm_gb_v2[0, 0] + cm_gb_v2[0, 1])
print("True positive rate", TPR_gb_v2)

TNR_gb_v2 = cm_gb_v2[1,1] / (cm_gb_v2[1,0] + cm_gb_v2[1,1])
print("True negative rate", TNR_gb_v2)

print("True accuracy rate", accuracy_score(solution_df["churn_probability"].to_numpy(), y_pred_actual))

**Let's recap the confusion matrix during model building**

In [ ]:
cm_gb = confusion_matrix(y_test, y_pred_gb)
print('The confusion Matrix : \n',cm_gb)

FPR_gb = cm_gb[1,0] / (cm_gb[1,0] + cm_gb[1,1])
print("False positive rate", FPR_gb)

TPR_gb = cm_gb[0, 0] / (cm_gb[0, 0] + cm_gb[0, 1])
print("True positive rate", TPR_gb)

TNR_gb = cm_gb[1,1] / (cm_gb[1,0] + cm_gb[1,1])
print("True negative rate", TNR_gb)

print(f"Training Score: {clf.score(x_train , y_train)}")
print(f"Test Score: {clf.score(x_test , y_test)}")

The drop in all compared metrics on test data set is not significant, so we can conclude the model is robust.

## Identifying significant features

In [ ]:
clf.feature_importances_

In [ ]:
significant_features = [] ## this empty list contains significant features shortly

## iterate over the list of feature importances
## if an item returns 0, the item is insignificant
for i in range(len(clf.feature_importances_)):
    if clf.feature_importances_[i] > 0:
        significant_features.append(i)

In [ ]:
significant_features

In [ ]:
## list of significant features

actual_df.columns[significant_features].to_list()

In [ ]:
solution_df.to_csv("Submission.csv")